In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np
import os
from matplotlib import pyplot as plt

In [2]:
pTime, cTime = 0, 0

CAMERA_INDEX = 0
CAMERA_WIDTH = 1920
CAMERA_HEIGHT = 1080
DATA_DURATION = 60
DATA_LENGTH = 30
DATA_BASE_PATH = 'raw_video'
CLASS_TAKEN = 'a'


In [3]:
try:
    os.makedirs(os.path.join(DATA_BASE_PATH, CLASS_TAKEN))
except:
    pass


In [4]:
def show_fps(image, pTime):
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(image, str(int(fps)), (10, 70),
                cv2.FONT_HERSHEY_PLAIN, 3, (255, 128, 255), 3)
    return image, pTime

def save_video(frames, filename):
    # We need to set resolutions.
    # So, convert them from float to integer.
    frame_width = CAMERA_WIDTH
    frame_height = CAMERA_HEIGHT

    size = (frame_width, frame_height)

    # Below VideoWriter object will create
    # a frame of above defined size.
    result = cv2.VideoWriter(filename,
                             cv2.VideoWriter_fourcc(*'mp4v'),
                             24, size)

    for frame in frames:
        # Write the frame into the video file
        result.write(frame)

    # Release the video write object
    result.release()

In [9]:
cap = cv2.VideoCapture(CAMERA_INDEX, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAMERA_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAMERA_HEIGHT)


if not cap.isOpened():
    print("Cannot open camera")
    exit()

frame_count = 0
capture_frames = False
session_count = 0
frames = []
video_counter = 1

while True:
    success, frame = cap.read()
    if not success:
        break

    frame_fps, pTime = show_fps(frame, pTime)
    key = cv2.waitKey(1) & 0xFF

    # If pressed space bar create a path for new dataset
    if key == ord(' '):
        frame_count = 0
        save_video(frames[-DATA_DURATION:], f'vide/video{video_counter}.mp4')
        video_counter += 1

        while(os.path.exists(os.path.join(DATA_BASE_PATH, CLASS_TAKEN))):
            session_count += 1

        if not os.path.exists(os.path.join(DATA_BASE_PATH, CLASS_TAKEN, str(session_count))):
            os.makedirs(os.path.join(DATA_BASE_PATH,
                        CLASS_TAKEN, str(session_count)))
            # session_count += 1

        capture_frames = True

    if capture_frames:
        frame_count += 1

        # Stop capturing frames after capturing 30 frames
        if frame_count == 30:
            capture_frames = False
            print("Finished capturing frames.")
            result.release()

    cv2.imshow("Holistic Mediapipe", frame)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


: 

: 